In [51]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import urllib2
import sqlite3

url = "http://db.netkeiba.com/race/201606050903/"
soup = BeautifulSoup(urllib2.urlopen(url), "html.parser")


In [52]:
dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

drop_table = '''drop table race'''
c.execute(drop_table)
drop_table = '''drop table race_by_horse'''
c.execute(drop_table)

c.close()

In [53]:
dbname = 'database.db'

conn = sqlite3.connect(dbname)
c = conn.cursor()

create_table = '''create table race (race_id int PRIMARY_KEY, title varchar(64),
                  place_id int, condition varchar(64), base_info varchar(64))'''
c.execute(create_table)

create_table = '''create table race_by_horse (rbh_id integer PRIMARY KEY, horse_id int, race_id int,
                  finished int, waku_ban int, uma_ban int, sex int, age int,
                  jockey_weight int, jockey_id int,
                  time varchar(64), odds, popularity int, weight int, old_flag int)'''
c.execute(create_table)

c.close()

In [54]:
race_title = soup.find("h1").text
race_condition = soup.find("diary_snap_cut").find("span").text
race_base_info = soup.find("p", {"class":"smalltxt"}).text

#year/place/number/day/xR
race_url = url.split("/")[-2]
print race_url
print race_title
print race_condition
print race_base_info

place_id = race_url[4:6]
print place_id

race_info = (race_url, race_title, place_id, race_condition, race_base_info)

201606050903
2歳未勝利
芝右 外1600m / 天候 : 曇 / 芝 : 良 / 発走 : 11:05
2016年12月25日 5回中山9日目 サラ系2歳未勝利  ○混□指(馬齢)
06


In [55]:
# -*- coding: utf-8 -*-
dbname = 'database.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()

insert_sql = """insert into race (race_id, title, place_id, condition, base_info)
          values (?,?,?,?,?)"""
c.execute(insert_sql, race_info)
conn.commit()

conn.close()

In [56]:
dbname = 'database.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()

select_sql ="select * from race"
results = c.execute(select_sql)
for res in results:
    print "----------"
    for r in res:
        print r

c.close()

----------
201606050903
2歳未勝利
6
芝右 外1600m / 天候 : 曇 / 芝 : 良 / 発走 : 11:05
2016年12月25日 5回中山9日目 サラ系2歳未勝利  ○混□指(馬齢)


In [57]:
# -*- coding: utf-8 -*-
main_table = soup.find("table", {"class":"race_table_01"})
tr_list = main_table.find_all("tr")

row_list = []

for i, tr in enumerate(tr_list):
    if i == 0:
        continue
    td_list = tr.findAll("td")
    finished = td_list[0].text
    waku_ban= td_list[1].text
    uma_ban = td_list[2].text
    horse_id = td_list[3].find("a")["href"].split("/")[2]
    sex_age = td_list[4].text
    sex = list(sex_age)[0]
    age = list(sex_age)[1]
    jockey_weight = td_list[5].text
    jockey_id = td_list[6].find("a")["href"].split("/")[2]
    time = td_list[7].text
    #diff, timeindex, passed, halftime is skipped
    odds = td_list[12].text
    popularity = td_list[13].text
    weight= td_list[14].text.split("(")[0]    
    row = (horse_id, race_url, finished, waku_ban, uma_ban, sex, age,
          jockey_weight, jockey_id, unicode(time), odds, popularity, weight, 0)
    row_list.append(row)
    
    #for r in row:
    #   print r
print "done race_by_horse"

done race_by_horse


In [58]:
# -*- coding: utf-8 -*-
dbname = 'database.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()

insert_sql = """insert into race_by_horse (horse_id, race_id, finished, waku_ban, uma_ban, sex, age,
          jockey_weight, jockey_id, time, odds, popularity, weight, old_flag)
          values (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""
c.executemany(insert_sql, row_list)
conn.commit()

conn.close()

In [59]:
# -*- coding: utf-8 -*-
dbname = 'database.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()

#select_sql = "select * from race_by_horse"
#results = c.execute(select_sql)
#for res in results:
#    print res
#print "-----------------"

param =(1,)
select_sql ="select * from race_by_horse where finished = ?"
results = c.execute(select_sql, param)
for res in results:
    print "-------"
    for r in res:
        print r,

c.close()

-------
1 2014101029 201606050903 1 2 4 牡 2 55 5203 1:37.7 16.8 8 464 0
